In [0]:
import pandas as pd  
import numpy as np
import matplotlib.pyplot as plt
import xgboost
plt.style.use('fivethirtyeight')

from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [9]:
from google.colab import files 
uploaded = files.upload() 

Saving 1105_news_data_preprocessed_labeled.csv to 1105_news_data_preprocessed_labeled.csv


In [82]:
uploaded = files.upload()

Saving wiki-news-300d-1M.vec to wiki-news-300d-1M (1).vec


KeyboardInterrupt: ignored

In [15]:
uploaded = files.upload() 

Saving news_forecast.csv to news_forecast.csv


In [11]:
uploaded = files.upload() 

Saving tickers_list.csv to tickers_list.csv


In [5]:
uploaded = files.upload() 

Saving news_scraping_results_and_label.csv to news_scraping_results_and_label.csv


In [0]:
old_news = pd.read_csv('./news_scraping_results_and_label.csv')
new_news = pd.read_csv('./1105_news_data_preprocessed_labeled.csv')
news = old_news.append(new_news, ignore_index=True)
news = news.drop(['sentiment'], axis=1)
tickers_list = pd.read_csv("./tickers_list.csv")

In [0]:
news_forecast = pd.read_csv('news_forecast.csv', index_col=0)

In [0]:
def label_sentiment(val, negative=-4.0, positive=4.0):
    if pd.isna(val): 
        return np.nan
    else: 
        if val <= negative: 
            return 0
        elif val >= positive:
            return 2
        else:
            return 1


news_forecast['sentiment'] = news_forecast['percent_change'].apply(lambda x: label_sentiment(x))
#news_forecast['sentiment'] = news_forecast['sentiment'].apply(lambda x: int(x))

In [0]:
news = news.join(news_forecast[['percent_change','sentiment']])
news = news.dropna(subset=['sentiment'])
news['sentiment'] = news['sentiment'].apply(lambda x: int(x))

In [0]:
suspended_ticker_news = news[news['SYMBOL'].isin(tickers_list['Symbol'].values)].reset_index()
active_ticker_news = news[~news['SYMBOL'].isin(tickers_list['Symbol'].values)].reset_index()

In [0]:
balanced_suspended = suspended_ticker_news[suspended_ticker_news['sentiment'] == 2].reset_index()
balanced_suspended = balanced_suspended.append(suspended_ticker_news[suspended_ticker_news['sentiment'] == 0].reset_index()[:71], ignore_index=True)
balanced_suspended = balanced_suspended.append(suspended_ticker_news[suspended_ticker_news['sentiment'] == 1].reset_index()[:71], ignore_index=True)

In [0]:
balanced_active = active_ticker_news[active_ticker_news['sentiment'] == 0].reset_index()
balanced_active = balanced_active.append(active_ticker_news[active_ticker_news['sentiment'] == 2].reset_index()[:1705], ignore_index=True)
balanced_active = balanced_active.append(active_ticker_news[active_ticker_news['sentiment'] == 1].reset_index()[:1705], ignore_index=True)

In [0]:
balanced_active['sentiment'].value_counts()

In [0]:
balanced_suspended_active = balanced_active.append(balanced_suspended, ignore_index=True)

In [0]:
corpus=balanced_active['CONTENT'].append(balanced_suspended['CONTENT'])

In [0]:
x = balanced_active['CONTENT']
y = balanced_active['sentiment']
x_test = balanced_suspended['CONTENT']
y_test = balanced_suspended['sentiment']

In [0]:
import sklearn.model_selection
from sklearn.model_selection import train_test_split
SEED = 2000
x_train, x_validation, y_train, y_validation = train_test_split(x, y, test_size=0.2, random_state=SEED)
#print (x_train)
y_train=y_train.astype('int')
y_validation=y_validation.astype('int')
y_test=y_test.astype('int')

In [29]:
# create a count vectorizer object 
count_vect = CountVectorizer()
count_vect.fit(x_train)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [0]:
# transform the training and validation data using count vectorizer object
xtrain_count =  count_vect.transform(x_train)
xtest_count =  count_vect.transform(x_validation)

In [0]:
xstest_count =  count_vect.transform(x_test)

In [0]:
##TODO: test max number of features work best
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000, min_df=50)
tfidf_vect.fit(x_train)
xtrain_tfidf =  tfidf_vect.transform(x_train)
xtest_tfidf =  tfidf_vect.transform(x_validation)

In [39]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=1000)
svd.fit(tfidf_vect.fit_transform(x_train))
xtrain_svd =  svd.transform(x_train)
xtest_svd =  svd.transform(x_validation)

ValueError: ignored

In [0]:
# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=None)
tfidf_vect_ngram.fit(x_train)
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(x_train)
xtest_tfidf_ngram =  tfidf_vect_ngram.transform(x_validation)

In [0]:
# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=None)
tfidf_vect_ngram_chars.fit(x_train)
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(x_train) 
xtest_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(x_validation) 

In [0]:
xstest_tfidf =  tfidf_vect.transform(x_test)
xstest_tfidf_ngram =  tfidf_vect_ngram.transform(x_test)
xstest_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(x_test)

In [80]:
# load the pre-trained word-embedding vectors 
embeddings_index = {}
for i, line in enumerate(open('./wiki-news-300d-1M.vec')):
    values = line.split()
    embeddings_index[values[0]] = np.asarray(values[1:], dtype='float32')

# create a tokenizer 
token = text.Tokenizer()
token.fit_on_texts(X)

word_index = token.word_index

# convert text to sequence of tokens and pad them to ensure equal length vectors 
train_seq_x = sequence.pad_sequences(token.texts_to_sequences(train_X), maxlen=70)
test_seq_x = sequence.pad_sequences(token.texts_to_sequences(test_X), maxlen=70)

# create token-embedding mapping
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

FileNotFoundError: ignored

In [0]:
test_seq_xs = sequence.pad_sequences(token.texts_to_sequences(Xs), maxlen=70)

In [0]:
def train_model(classifier, feature_vector_train, label, feature_vector_test, feature_vector_suspended, is_neural_net=False, classifier_name=None, feature_vector_name=None):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_test)
    
    # predict the labels on the suspension stock dataset
    suspended_prediction = classifier.predict(feature_vector_suspended)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
        suspended_prediction = suspended_prediction.argmax(axis=-1)
    
    accuracy = metrics.accuracy_score(y_validation, predictions)
    precision = metrics.precision_score(y_validation, predictions, average=None)
    recall = metrics.recall_score(y_validation, predictions, average=None)
    f1 = metrics.f1_score(y_validation, predictions, average=None)
    
    
    suspended_accuracy = metrics.accuracy_score(y_test, suspended_prediction)
    suspended_precision = metrics.precision_score(y_test, suspended_prediction, average=None)
    suspended_recall = metrics.recall_score(y_test, suspended_prediction, average=None)
    suspended_f1 = metrics.f1_score(y_test, suspended_prediction, average=None)
    
    df = pd.DataFrame([[classifier_name, feature_vector_name, accuracy,precision,recall,f1, suspended_accuracy, suspended_precision, suspended_recall, suspended_f1]], columns=['classifier','feature vector','accuracy','precision','recall','f1', 'suspended_accuracy', 'suspended_precision', 'suspended_recall', 'suspended_f1'])
    
    return df

In [0]:
metrics_df = pd.DataFrame(columns=['classifier', 'feature vector', 'accuracy', 'precision', 'recall', 'f1', 'suspended_accuracy', 'suspended_precision', 'suspended_recall', 'suspended_f1'])

In [0]:
# Naive Bayes on Count Vectors
scores = train_model(naive_bayes.MultinomialNB(), xtrain_count, y_train, xtest_count, xstest_count, classifier_name='Naive Bayes', feature_vector_name='Count')
metrics_df = metrics_df.append(scores, ignore_index=True)

In [0]:
# Naive Bayes on word level TF-IDF Vectors
scores = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf, y_train, xtest_tfidf, xstest_tfidf, classifier_name='Naive Bayes', feature_vector_name='word TF-IDF')
metrics_df = metrics_df.append(scores, ignore_index=True)

In [0]:
# Naive Bayes on ngram level TF-IDF Vectors
scores = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, y_train, xtest_tfidf_ngram, xstest_tfidf_ngram, classifier_name='Naive Bayes', feature_vector_name='ngram TF-IDF')
metrics_df = metrics_df.append(scores, ignore_index=True)

In [62]:
# Naive Bayes on characters level TF-IDF Vectors
train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram_chars, y_train, xtest_tfidf_ngram_chars, xstest_tfidf_ngram_chars, classifier_name='Naive Bayes', feature_vector_name='chars TF-IDF')
metrics_df = metrics_df.append(scores, ignore_index=True)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [63]:
# Logistic Regression on Count Vectors
scores = train_model(linear_model.LogisticRegression(), xtrain_count, y_train, xtest_count, xstest_count, classifier_name='Logistic Regression', feature_vector_name='Count')
metrics_df = metrics_df.append(scores, ignore_index=True)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [64]:
# Logistic Regression on TF-IDF Vectors
scores = train_model(linear_model.LogisticRegression(), xtrain_tfidf, y_train, xtest_tfidf, xstest_tfidf, classifier_name='Logistic Regression', feature_vector_name='TF-IDF')
metrics_df = metrics_df.append(scores, ignore_index=True)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [65]:
# SVM on Ngram Level TF IDF Vectors
scores = train_model(svm.SVC(), xtrain_tfidf_ngram, y_train, xtest_tfidf_ngram, xstest_tfidf_ngram, classifier_name='SVM', feature_vector_name='ngram TF-IDF')
metrics_df = metrics_df.append(scores, ignore_index=True)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [66]:
# Random Forest on Count Vectors
scores = train_model(ensemble.RandomForestClassifier(), xtrain_count, y_train, xtest_count, xstest_count, classifier_name='Random Forest', feature_vector_name='Count')
metrics_df = metrics_df.append(scores, ignore_index=True)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [67]:
# Random Forest on TF-IDF Vectors
scores = train_model(ensemble.RandomForestClassifier(), xtrain_tfidf, y_train, xtest_tfidf, xstest_tfidf, classifier_name='Random Forest', feature_vector_name='word TF-IDF')
metrics_df = metrics_df.append(scores, ignore_index=True)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [0]:
# Extreme Boosting Gradient on Count Vectors
scores = train_model(xgboost.XGBClassifier(), xtrain_count.tocsc(), y_train, xtest_count.tocsc(), xstest_count.tocsc(), classifier_name='Extreme Boosting Gradient', feature_vector_name='Count')
metrics_df = metrics_df.append(scores, ignore_index=True)

In [0]:
# Extreme Boosting Gradient on TF-IDF Vectors
scores = train_model(xgboost.XGBClassifier(), xtrain_tfidf.tocsc(), y_train, xtest_tfidf.tocsc(), xstest_tfidf.tocsc(), classifier_name='Extreme Gradient Boosting', feature_vector_name='word TF-IDF')
metrics_df = metrics_df.append(scores, ignore_index=True)

In [76]:
metrics_df

,classifier,feature vector,accuracy,precision,recall,f1,suspended_accuracy,suspended_precision,suspended_recall,suspended_f1
0,Naive Bayes,Count,0.579668,"[0.5729537366548043, 0.5601659751037344, 0.623...","[0.4791666666666667, 0.8157099697885196, 0.455...","[0.5218800648298216, 0.6642066420664208, 0.525...",0.338028,"[0.42857142857142855, 0.31724137931034485, 0.3...","[0.2535211267605634, 0.647887323943662, 0.1126...","[0.3185840707964602, 0.425925925925926, 0.1649..."
1,Naive Bayes,word TF-IDF,0.513196,"[0.5245283018867924, 0.48541666666666666, 0.55...","[0.41369047619047616, 0.7039274924471299, 0.42...","[0.46256239600665555, 0.5745992601726264, 0.48...",0.300469,"[0.3090909090909091, 0.2857142857142857, 0.34375]","[0.23943661971830985, 0.5070422535211268, 0.15...","[0.2698412698412698, 0.365482233502538, 0.2135..."
2,Naive Bayes,ngram TF-IDF,0.568915,"[0.5333333333333333, 0.5522041763341067, 0.635...","[0.5, 0.7190332326283988, 0.4943820224719101]","[0.5161290322580646, 0.6246719160104988, 0.556...",0.305164,"[0.3404255319148936, 0.26865671641791045, 0.40...","[0.22535211267605634, 0.5070422535211268, 0.18...","[0.27118644067796616, 0.35121951219512193, 0.2..."
3,Naive Bayes,ngram TF-IDF,0.568915,"[0.5333333333333333, 0.5522041763341067, 0.635...","[0.5, 0.7190332326283988, 0.4943820224719101]","[0.5161290322580646, 0.6246719160104988, 0.556...",0.305164,"[0.3404255319148936, 0.26865671641791045, 0.40...","[0.22535211267605634, 0.5070422535211268, 0.18...","[0.27118644067796616, 0.35121951219512193, 0.2..."
4,Logistic Regression,Count,0.594330,"[0.5552407932011332, 0.6910828025477707, 0.547...","[0.5833333333333334, 0.6555891238670695, 0.547...","[0.5689404934687953, 0.6728682170542635, 0.547...",0.356808,"[0.3770491803278688, 0.32941176470588235, 0.37...","[0.323943661971831, 0.39436619718309857, 0.352...","[0.34848484848484845, 0.358974358974359, 0.362..."
5,Logistic Regression,TF-IDF,0.583578,"[0.553125, 0.6060606060606061, 0.5882352941176...","[0.5267857142857143, 0.6646525679758308, 0.561...","[0.5396341463414634, 0.6340057636887607, 0.574...",0.295775,"[0.3, 0.27184466019417475, 0.35]","[0.29577464788732394, 0.39436619718309857, 0.1...","[0.2978723404255319, 0.3218390804597701, 0.252..."
6,SVM,ngram TF-IDF,0.323558,"[0.0, 0.3235581622678397, 0.0]","[0.0, 1.0, 0.0]","[0.0, 0.48892171344165436, 0.0]",0.333333,"[0.0, 0.3333333333333333, 0.0]","[0.0, 1.0, 0.0]","[0.0, 0.5, 0.0]"
7,Random Forest,Count,0.495601,"[0.4753623188405797, 0.5176151761517616, 0.491...","[0.4880952380952381, 0.5770392749244713, 0.426...","[0.4816446402349486, 0.5457142857142857, 0.457...",0.333333,"[0.38666666666666666, 0.27835051546391754, 0.3...","[0.4084507042253521, 0.38028169014084506, 0.21...","[0.3972602739726028, 0.3214285714285714, 0.267..."
8,Random Forest,word TF-IDF,0.505376,"[0.45077720207253885, 0.5545977011494253, 0.51...","[0.5178571428571429, 0.5830815709969789, 0.421...","[0.4819944598337951, 0.5684830633284242, 0.465...",0.286385,"[0.3333333333333333, 0.26881720430107525, 0.25...","[0.323943661971831, 0.352112676056338, 0.18309...","[0.3285714285714286, 0.3048780487804878, 0.213..."
9,Extreme Boosting Gradient,Count,0.575758,"[0.5359477124183006, 0.5799086757990868, 0.612...","[0.4880952380952381, 0.7673716012084593, 0.480...","[0.5109034267912772, 0.6605981794538361, 0.538...",0.314554,"[0.40476190476190477, 0.2826086956521739, 0.33...","[0.23943661971830985, 0.5492957746478874, 0.15...","[0.30088495575221236, 0.3732057416267942, 0.21..."


In [0]:
def create_rnn_lstm():
    # Add an Input Layer
    input_layer = layers.Input((70, ))
    #input_layer = layers.Input(X.shape)
    
    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=True)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the LSTM Layer
    lstm_layer = layers.LSTM(100)(embedding_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(lstm_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(3, activation="softmax")(lstm_layer)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model


In [0]:
classifier = create_rnn_lstm()
#accuracy = train_model(classifier, train_seq_x, y_train, test_seq_x, test_seq_xs, is_neural_net=True, classifier_name='LSTM', feature_vector_name='fastText embedding')
#print("RNN-LSTM, Word Embeddings",  accuracy)
classifier.fit(train_seq_x, train_y)

In [0]:
predictions = classifier.predict(test_seq_x)

In [0]:
x_train_tfidf = TfidfVectorizer(max_features=60000,ngram_range=(1, 3)).fit_transform(x_train)

In [0]:
from sklearn.feature_selection import chi2
import matplotlib.pyplot as plt
chi2score = chi2(x_train_tfidf, y_train)[0]

In [0]:
wscores = list(zip(tfidf_vect.get_feature_names(), chi2score))
wchi2 = sorted(wscores, key=lambda x:x[1])
topchi2 = list(zip(*wchi2[-20:]))
x = range(len(topchi2[1]))
labels = topchi2[0]
plt.figure(figsize=(15,10))
plt.barh(x,topchi2[1], align='center', alpha=0.2)
plt.plot(topchi2[1], x, '-o', markersize=5, alpha=0.8)
plt.yticks(x, labels)
plt.xlabel('$\chi^2$')

In [0]:
from sklearn.feature_selection import SelectKBest, chi2
ch2_result = []
for n in np.arange(10000,70000,10000):
    ch2 = SelectKBest(chi2, k=n)
    x_train_chi2_selected = ch2.fit_transform(xtrain_tfidf, y_train)
    x_validation_chi2_selected = ch2.transform(xstest_tfidf)
    clf = linear_model.LogisticRegression()
    clf.fit(x_train_chi2_selected, y_train)
    score = clf.score(x_validation_chi2_selected, test_ys)
    ch2_result.append(score)
    print("chi2 feature selection evaluation calculated for {} features".format(n))

In [92]:
ch2_result

[]

In [0]:
plt.figure(figsize=(8,6))
#plt.plot(nfeatures_plot_tgt.nfeatures, nfeatures_plot_tgt.validation_accuracy,label='trigram tfidf vectorizer',color='royalblue')
plt.plot(np.arange(10000,70000,10000), ch2_result,label='tfidf dimesions reduced from 60000 features',linestyle=':', color='orangered')

plt.title("tfidft vectorizer: features limited within tfidft vectorizer VS reduced dimensions with chi2")
plt.xlabel("Number of features")
plt.ylabel("Validation set accuracy")
plt.legend()